# Full solution for $d=4$, $q\ge 5$

In [ ]:
# Load code in external files
load('./local_views.pyx')
load('./properties.pyx')

# Set up variables common to all computations.
q, lam, r, t = var('q, lam, r, t')

d       = 4
lam_sub = 1/(1+t)

# The number of processes to spawn in parallel
# -1 means the number of CPUs (including hyperthreads)
# -2 means all but one.
n_jobs = -2

print "Done."

## Generating the local views and calculating properties

Needs to be done only once as data is saved to disk. This takes a long time.

In [ ]:
save_Ls(d)
save_all_props(d, n_jobs)

## Minimization

- $q\le4$ cannot be solved with this method: the LP seems to be underconstrained.
- $q=5$ and $q\ge6$ are done separately.
- As with the $d=3$ paper, we substitute variables and show that the slack is a polynomial with positive coefficients in the relevant variables.

We use $e^{-b} = \lambda = 1/(1+t)$, and in the $q\ge 6$ computation we use $q=r+6$.

### Minimization for $q=5$

We set $q=5$ and use constraints corresponding to partitions $4$ and $2+1+1$ of $4$.

In [ ]:
q_sub = 5
Ss = map(Partition, [[4], [2, 1, 1]])

The method depends on a positive 'magic factor' which clears the denominator in every slack.

In [ ]:
min_magic_q5 = 6*(16*t^10 + 176*t^9 + 888*t^8 + 2676*t^7 + 5309*t^6 + 
                  7260*t^5 + 6996*t^4 + 4760*t^3 + 2224*t^2 + 660*t + 
                  100)*(2*t^2 + 6*t + 5)*(t + 1)^31*t^2

# Set up some variables for the slack function

fields = ['Z', 'ZUv'] + [('Zgamd', Partition(S)) for S in Ss]

ZKdd_orig  = build_ZKdd(d, q, lam)
ZUKdd_orig = lam*ZKdd_orig.derivative(lam)/(2*d)
ZKdd_sub   = ZKdd_orig.subs(q=q_sub, lam=lam_sub)
ZUKdd_sub  = ZUKdd_orig.subs(q=q_sub, lam=lam_sub)
ps_Kdd_sub = list(gen_props_Kdd(d, q_sub, lam_sub))
ZDelta = build_ZDelta(d, ZKdd_sub, ZUKdd_sub, Ss, ps_Kdd_sub)

def slack(p, magic):
    if p['isKdd']:
        return 0
    slack = (ZKdd_sub*p['ZUv'] - p['Z']*ZUKdd_sub
             - sum(ZDelta[S]*p['Zgamd', S] for S in Ss)) * magic
    return slack.polynomial(ZZ)

# A test function that return True if the slack has the desired properties
def succeeds(p):
    if p['isKdd']:
        return True
    
    subs_props(p, q_sub, lam_sub, fields)
    m = slack(p, min_magic_q5)

    return all(c >= 0 for c in m.coefficients())

In [ ]:
# Run the tests in parallel, collecting the results in a list
aprun = TqdmParallelExecutor(n_jobs=n_jobs)
results = aprun(total=int(num_Ls[d]), leave=True)(delayed(succeeds)(p) 
                                                  for p in read_all_props(d))

if all(results):
    print('Success!')
else:
    print('Fail!')

### Minimization for $q\ge6$

We set $q=r+6$ and use constraints corresponding to partitions $2+1+1$ and $1+1+1+1$ of $4$.

In [ ]:
q_sub = r+6
Ss = map(Partition, [[2, 1, 1], [1, 1, 1, 1]])

The method depends on a *different* positive 'magic factor' which clears the denominator in every slack.

In [ ]:
min_magic = 2*(r^4*t^9 + 9*r^4*t^8 + 11*r^3*t^9 + 36*r^4*t^7 + 105*r^3*t^8 + 
               39*r^2*t^9 + 84*r^4*t^6 + 447*r^3*t^7 + 411*r^2*t^8 + 51*r*t^9 + 
               126*r^4*t^5 + 1117*r^3*t^6 + 1926*r^2*t^7 + 645*r*t^8 + 18*t^9 + 
               126*r^4*t^4 + 1809*r^3*t^5 + 5286*r^2*t^6 + 3477*r*t^7 + 
               342*t^8 + 84*r^4*t^3 + 1971*r^3*t^4 + 9393*r^2*t^5 + 10715*r*t^6 + 
               2250*t^7 + 36*r^4*t^2 + 1445*r^3*t^3 + 11241*r^2*t^4 + 21096*r*t^5 + 
               7954*t^6 + 9*r^4*t + 687*r^3*t^2 + 9090*r^2*t^3 + 27816*r*t^4 + 
               17508*t^5 + r^4 + 192*r^3*t + 4806*r^2*t^2 + 24812*r*t^3 + 
               25428*t^4 + 24*r^3 + 1512*r^2*t + 14580*r*t^2 + 24888*t^3 + 
               216*r^2 + 5184*r*t + 16200*t^2 + 864*r + 6480*t + 
               1296)*(r + 4)*(r + 3)*(t + 1)^35

In [ ]:
# Set up some variables for the slack function

fields = ['Z', 'ZUv'] + [('Zgamd', Partition(S)) for S in Ss]

ZKdd_orig  = build_ZKdd(d, q, lam)
ZUKdd_orig = lam*ZKdd_orig.derivative(lam)/(2*d)
ZKdd_sub   = ZKdd_orig.subs(q=q_sub, lam=lam_sub)
ZUKdd_sub  = ZUKdd_orig.subs(q=q_sub, lam=lam_sub)
ps_Kdd_sub = list(gen_props_Kdd(d, q_sub, lam_sub))
ZDelta = build_ZDelta(d, ZKdd_sub, ZUKdd_sub, Ss, ps_Kdd_sub)

def slack(p, magic):
    if p['isKdd']:
        return 0
    slack = (ZKdd_sub*p['ZUv'] - p['Z']*ZUKdd_sub
             - sum(ZDelta[S]*p['Zgamd', S] for S in Ss)) * magic
    return slack.polynomial(ZZ)

# A test function that return True if the slack has the desired properties
def succeeds(p):
    if p['isKdd']:
        return True
    
    subs_props(p, q_sub, lam_sub, fields)
    m = slack(p, min_magic)

    return all(c >= 0 for c in m.coefficients())

In [ ]:
# Run the tests in parallel, collecting the results in a list
aprun = TqdmParallelExecutor(n_jobs=n_jobs)
results = aprun(total=int(num_Ls[d]), leave=True)(delayed(succeeds)(p)
                                                  for p in read_all_props(d))

if all(results):
    print('Success!')
else:
    print('Fail!')

## Maximization

- $q\le4$ cannot be solved with this method.
- The whole range $q\ge5$ is done with one calculation.
- As with the $d=3$ paper, we substitute variables and show that the slack is a polynomial with positive coefficients in the relevant variables.

We use $e^{-b} = \lambda = 1/(1+t)$, and $q=r+5$. A magic factor is also present like above.

In [ ]:
q_sub = r+5
max_magic = 2*(t + 1)^25

In [ ]:
# Set up some variables for the slack function
q_sub   = r+5

fields = ['Z', 'ZUv']

ZKd1_orig  = build_ZKd1(d, q, lam)
ZUKd1_orig = lam*ZKd1_orig.derivative(lam)/(d+1)
ZKd1_sub   = ZKd1_orig.subs(q=q_sub, lam=lam_sub)
ZUKd1_sub  = ZUKd1_orig.subs(q=q_sub, lam=lam_sub)

def slack(p, magic):
    if p['isKd1']:
        return 0
    slack = (p['Z']*ZUKd1_sub - ZKd1_sub*p['ZUv']) * magic
    return slack.polynomial(ZZ)

# A test function that return True if the slack has the desired properties
def succeeds(p):
    if p['isKd1']:
        return True
    
    subs_props(p, q_sub, lam_sub, fields)
    m = slack(p, max_magic)

    return all(c >= 0 for c in m.coefficients())

In [ ]:
# Run the tests in parallel, collecting the results in a list
aprun = TqdmParallelExecutor(n_jobs=n_jobs)
results = aprun(total=int(num_Ls[d]), leave=True)(delayed(succeeds)(p) 
                                                  for p in read_all_props(d))

if all(results):
    print('Success!')
else:
    print('Fail!')